In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [29]:
model = BayesianNetwork([('B', 'A'), ('E', 'A'), ('E', 'R')])

cpd_b = TabularCPD(variable='B', variable_card=2, values=[[1-0.01], [0.01]])
cpd_e = TabularCPD(variable='E', variable_card=2, values=[[1-10**-6], [10**-6]])
cpd_r = TabularCPD(
                variable='R', variable_card=2,
                #      e=f  e=t
                values=[[1, 0],  # r = f
                        [0, 1]], # r = t
                evidence=['E'],
                evidence_card=[2])
cpd_a = TabularCPD(
                variable='A', variable_card=2,
                # B|E=     ff       ft      tf      tt 
                values=[[1-0.001, 1-0.41, 1-0.95, 1-0.98], # a = f
                        [0.001,     0.41,   0.95,   0.98]],# a = t
                evidence=['B', 'E'],
                evidence_card=[2, 2])

model.add_cpds(cpd_b, cpd_e, cpd_r, cpd_a)
model.check_model()


infer = VariableElimination(model)
print(f"P(A=t)={round(infer.query(['A']).values[1], 7)}")
print(f"P(A=t|R=t)={round(infer.query(['A'], {'R':1}).values[1], 7)}")
print(f"P(B=t|A=t)={round(infer.query(['B'], {'A':1}).values[1], 6)}")
print(f"P(B=t|A=t,R=t)={round(infer.query(['B'], {'A':1,'R':1}).values[1], 6)}")

P(A=t)=0.0104904
P(A=t|R=t)=0.4157
P(B=t|A=t)=0.905589
P(B=t|A=t,R=t)=0.023575
